# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [1]:
sc

Calculation started (calculation_id=28c5c006-ac1c-73da-abe1-552dc5cf794a) in (session=9ec5c006-8acd-9c67-3b8e-64ff2df3df0a). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
<SparkContext master=athena appName=default>



<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [43]:
#export
from pyspark.sql.functions import *
from pyspark.sql import *

Calculation started (calculation_id=ccc5c018-7dd9-3350-11a2-fc0e71d4d0df) in (session=9ec5c006-8acd-9c67-3b8e-64ff2df3df0a). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [3]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

Calculation started (calculation_id=38c5c006-b76c-f705-cf93-41eb12ae20f2) in (session=9ec5c006-8acd-9c67-3b8e-64ff2df3df0a). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [4]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'ndinapoli6'

Calculation started (calculation_id=aec5c006-b8f0-93b0-e895-f675b529104d) in (session=9ec5c006-8acd-9c67-3b8e-64ff2df3df0a). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [5]:
#export
def long_trips(trips):
    # Returns a Dataframe (trips) with Schema the same as :trips:
    long_trips_df = trips.filter(col("trip_distance") >= 2)
    
    return long_trips_df

Calculation started (calculation_id=84c5c006-ba1e-f714-ef40-75c79ba65467) in (session=9ec5c006-8acd-9c67-3b8e-64ff2df3df0a). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [6]:
#export
def manhattan_trips(trips, lookup):
    
    trips_borough = trips.join(lookup, trips.DOLocationID == lookup.LocationID, "inner")
    manhattan_trips_df = trips_borough.filter(col("Borough") == "Manhattan")

    aggregated_df = manhattan_trips_df.groupBy("DOLocationID").agg(sum("passenger_count").alias("pcount"))

    windowSpec = Window.orderBy(col("pcount").desc())
    ranked_df = aggregated_df.withColumn("rank", dense_rank().over(windowSpec))

    top_locations = ranked_df.filter(col("rank") <= 20).drop("rank")

    return top_locations

Calculation started (calculation_id=70c5c006-bb58-659d-9f12-1dfa3cf114a1) in (session=9ec5c006-8acd-9c67-3b8e-64ff2df3df0a). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [53]:
#export
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    from pyspark.sql.types import DecimalType
    pickup_locations = trips.groupBy("PULocationID").agg(
        avg("total_amount").alias("avg_total_amount"),
        count("*").alias("total_trips")
    )

    pickup_locations_with_mtrips = pickup_locations.join(
        mtrips,
        pickup_locations.PULocationID == mtrips.DOLocationID,
        "left_outer"
    )

    window_spec = Window.partitionBy().orderBy()
    pickup_locations_with_mtrips = pickup_locations_with_mtrips.withColumn(
        "trips_to_popular_destinations",
        count(when(col("DOLocationID").isNotNull(), 1)).over(window_spec)
    )

    pickup_locations_with_mtrips = pickup_locations_with_mtrips.withColumn(
        "proportion_to_popular_destinations",
        col("trips_to_popular_destinations") / col("total_trips")
    )
    pickup_locations_with_mtrips = pickup_locations_with_mtrips.withColumn(
        "weighted_profit",
        col("avg_total_amount") * col("proportion_to_popular_destinations")
    )
    
    pickup_locations_with_mtrips = pickup_locations_with_mtrips.withColumn(
        "weighted_profit",
        col("weighted_profit").cast(DecimalType(38, 10))
    )

    result = pickup_locations_with_mtrips.select("PULocationID", "weighted_profit")

    return result


Calculation started (calculation_id=4ac5c01e-e43d-a7d4-f994-ae71e0e09449) in (session=9ec5c006-8acd-9c67-3b8e-64ff2df3df0a). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [55]:
#export
def final_output(wp, lookup): 
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    final_result = wp.join(
        lookup,
        wp.PULocationID == lookup.LocationID,
        "inner"
    )

    final_result = final_result.select(
        col("Zone").alias("Zone"),
        col("Borough").alias("Borough"),
        col("weighted_profit").alias("weighted_profit")
    )

    final_result = final_result.orderBy(desc("weighted_profit"))
    final_result = final_result.limit(20)

    return final_result

Calculation started (calculation_id=54c5c023-4352-e13e-67b8-5086b98d9fa0) in (session=9ec5c006-8acd-9c67-3b8e-64ff2df3df0a). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

In [9]:
trips, lookup = load_data()
trips.show(5)
lookup.show(5)

Calculation started (calculation_id=bac5c006-c739-b32a-9791-8015bd5b00b4) in (session=9ec5c006-8acd-9c67-3b8e-64ff2df3df0a). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
Trip Count:  7667792
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2019-01-01 00:46:40|  2019-01-01 00:53:20|              1|          1.5|         1|                 N|         151|         239|           1|        7.0|  0.5|    0.5| 

In [10]:
trips = long_trips(trips)
trips.show(5)

Calculation started (calculation_id=3cc5c007-1267-751a-2eb1-34f27e02212a) in (session=9ec5c006-8acd-9c67-3b8e-64ff2df3df0a). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2019-01-01 00:59:47|  2019-01-01 01:18:59|              1|          2.6|         1|                 N|         239|         246|           1|       14.0|  0.5|    0.5|       1.0|         0.

In [49]:
mtrips = manhattan_trips(trips, lookup)
mtrips.show()

Calculation started (calculation_id=06c5c01e-2e34-daf4-d66c-43ac13e0eed1) in (session=9ec5c006-8acd-9c67-3b8e-64ff2df3df0a). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+------------+------+
|DOLocationID|pcount|
+------------+------+
|         236|147086|
|         161|117642|
|         230|112155|
|         239|104164|
|         162|101209|
|         238|100189|
|          48| 98784|
|         170| 96109|
|         142| 95053|
|         231| 94621|
|         263| 94548|
|          79| 94421|
|         237| 90335|
|         141| 90061|
|         140| 84212|
|          68| 79559|
|         234| 75786|
|         186| 73863|
|         163| 73145|
|          13| 70693|
+------------+------+



In [56]:
wp = weighted_profit(trips, mtrips)

Calculation started (calculation_id=e0c5c023-725e-8f2d-5b85-388da7e2a293) in (session=9ec5c006-8acd-9c67-3b8e-64ff2df3df0a). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [57]:
final_output(wp, lookup).show()

Calculation started (calculation_id=dec5c023-817d-1df3-d35b-4be8fb0f6528) in (session=9ec5c006-8acd-9c67-3b8e-64ff2df3df0a). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+--------------------+-------------+---------------+
|                Zone|      Borough|weighted_profit|
+--------------------+-------------+---------------+
|       Rikers Island|        Bronx| 769.4000000000|
|Governor's Island...|    Manhattan| 631.2000000000|
|     Freshkills Park|Staten Island| 586.8500000000|
|       Port Richmond|Staten Island| 487.1555555556|
|       Arden Heights|Staten Island| 405.9250000000|
|Breezy Point/Fort...|       Queens| 396.0000000000|
|Arrochar/Fort Wad...|Staten Island| 317.8444444444|
|   Rossville/Woodrow|Staten Island| 295.0000000000|
|       West Brighton|Staten Island| 286.4888888889|
|       Broad Channel|       Queens| 268.1000000000|
|         Jamaica Bay|       Queens| 235.5277777778|
|             Oakwood|Staten Island| 216.0000000000|
|         Great Kills|Staten Island| 155.0666666667|
|         Westerleigh|Staten Island| 121.5560000000|
| Grymes Hill/Clifton|Staten Island| 110.7560000000|
|        Country Club| 

In [58]:
main('large','s3://cse6242-ndinapoli6/output-large1' )

Calculation started (calculation_id=70c5c026-48b8-07a0-0003-ac0c0c8c4383) in (session=9ec5c006-8acd-9c67-3b8e-64ff2df3df0a). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
ndinapoli6
Trip Count:  187203269
+--------------------+-------------+---------------+
|                Zone|      Borough|weighted_profit|
+--------------------+-------------+---------------+
|Governor's Island...|    Manhattan| 631.2000000000|
|    Great Kills Park|Staten Island| 116.7250000000|
|       Rikers Island|        Bronx|  82.0420000000|
|   Rossville/Woodrow|Staten Island|  66.5438271605|
|     Freshkills Park|Staten Island|  37.7469387755|
|Eltingville/Annad...|Staten Island|  25.9042534722|
|             Oakwood|Staten Island|  24.5232638889|
|         Great Kills|Staten Island|  23.6747041420|
|       Broad Channel|       Queens|  15.4981930027|
|       West Brighton|Staten Island|  14.0573347107|
|       Port Richmond|Staten Island|  11.0098530963|
|New Dorp/Midland ...|Staten Island|   9.7293733180|
|         Westerleigh|Staten Island|   8.7748204518|
|         Jamaica Bay|       Queens|   8.7695022804|
|Breezy Point/Fort...|       Queens|   6.8